# Data retrival and preprocessing

In [1]:
ticker = 'TATAMOTORS.NS'
num_of_days = 4

In [11]:
from models.MLPModel import MLPModel

model = MLPModel(ticker, num_of_days)


[*********************100%***********************]  1 of 1 downloaded


In [2]:
from models.LSTMModel import LSTMModel

model = LSTMModel(ticker, num_of_days)

Using TensorFlow backend.


[*********************100%***********************]  1 of 1 downloaded
Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.normalized_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-01-01,0.657293,0.691786,0.655377,0.690637,0.156121,116802.85
1997-01-02,0.695619,0.695619,0.676456,0.683642,0.154540,128507.66
1997-01-03,0.676456,0.689774,0.674827,0.687092,0.155320,79989.83
1997-01-06,0.680289,0.690349,0.672815,0.677414,0.153132,56131.43
1997-01-07,0.675977,0.675977,0.653939,0.659113,0.148995,114660.71
1997-01-08,0.669749,0.684121,0.665437,0.670036,0.151464,52462.92
1997-01-09,0.670036,0.675498,0.663042,0.667928,0.150987,56314.08
1997-01-10,0.674540,0.689678,0.672815,0.685175,0.154886,97257.43
1997-01-13,0.680289,0.680289,0.669366,0.676456,0.152915,42305.35


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 4, 512)            2560      
_________________________________________________________________
dense_3 (Dense)              (None, 4, 512)            262656    
_________________________________________________________________
dense_4 (Dense)              (None, 4, 512)            262656    
_________________________________________________________________
dense_5 (Dense)              (None, 4, 512)            262656    
_________________________________________________________________
dense_6 (Dense)              (None, 4, 128)            65664     
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 513       
Total para

In [14]:
model.compile()

In [15]:
model.fit(500)

Train on 2076 samples, validate on 2077 samples
Epoch 1/500
2076/2076 [==============================] - 2s 827us/step - loss: 0.0084 - mean_absolute_percentage_error: 11.7563 - mean_absolute_error: 0.0472 - val_loss: 0.0053 - val_mean_absolute_percentage_error: 3.5957 - val_mean_absolute_error: 0.0543

Epoch 00001: val_loss improved from inf to 0.00533, saving model to weights/TATAMOTORS.NS.weights.best.hdf5
Epoch 2/500
2076/2076 [==============================] - 1s 480us/step - loss: 5.5250e-04 - mean_absolute_percentage_error: 4.0425 - mean_absolute_error: 0.0167 - val_loss: 0.0047 - val_mean_absolute_percentage_error: 3.3457 - val_mean_absolute_error: 0.0506

Epoch 00002: val_loss improved from 0.00533 to 0.00468, saving model to weights/TATAMOTORS.NS.weights.best.hdf5
Epoch 3/500
2076/2076 [==============================] - 1s 477us/step - loss: 5.6138e-04 - mean_absolute_percentage_error: 4.1112 - mean_absolute_error: 0.0168 - val_loss: 0.0039 - val_mean_absolute_percentage_erro


Epoch 00050: val_loss did not improve from 0.00203
Epoch 51/500
2076/2076 [==============================] - 1s 444us/step - loss: 3.2867e-04 - mean_absolute_percentage_error: 3.1933 - mean_absolute_error: 0.0131 - val_loss: 0.0047 - val_mean_absolute_percentage_error: 3.2733 - val_mean_absolute_error: 0.0532

Epoch 00051: val_loss did not improve from 0.00203
Epoch 52/500
2076/2076 [==============================] - 1s 433us/step - loss: 3.0831e-04 - mean_absolute_percentage_error: 3.0740 - mean_absolute_error: 0.0126 - val_loss: 0.0049 - val_mean_absolute_percentage_error: 3.3990 - val_mean_absolute_error: 0.0543

Epoch 00052: val_loss did not improve from 0.00203
Epoch 53/500
2076/2076 [==============================] - 1s 448us/step - loss: 3.4057e-04 - mean_absolute_percentage_error: 3.2426 - mean_absolute_error: 0.0134 - val_loss: 0.0021 - val_mean_absolute_percentage_error: 2.2000 - val_mean_absolute_error: 0.0332

Epoch 00053: val_loss did not improve from 0.00203
Epoch 54/500

In [16]:
model.load_weights()

In [17]:
score = model.evaluate()

1039/1039 [==============================] - 0s 93us/step


In [18]:
score

[0.0094185136865889, 1.6834844949729633, 0.07332274798909785]

In [6]:
from models.KNNModel import KNNModel

model = KNNModel(ticker, num_of_days, normalization_factor=10)

y_pred = model.predict(model.x_train)

[*********************100%***********************]  1 of 1 downloaded


In [4]:
y_pred = model.predict(model.x_test)

In [5]:
y_pred

array([[3.10838384],
       [3.10838384],
       [3.10969697],
       ...,
       [3.10838384],
       [3.10838384],
       [3.10838384]])